# COGS 108 - Final Project 

# Overview

Our project sought to discern any quantifiable effects on the rate of change in violent crime from the installation of surveillance cameras in various neighborhoods of Chicago. This task was accomplished by comparing violent crime rates within neighborhoods with high crime rates before and after their installation. Overall, we found no evidence to suggest that surveillance cameras are effective way to curtail violent crimes.

# Names

- Chaitanya Patel
- Kristine Marie Baluyot
- Linh Le
- Namit Mishra
- Tiffany Zhang
- Robert Eaton

# Group Members IDs

- A15346478 
- A13447798 
- A14201350
- A92112718
- A13161270
- A14190293

# Research Question

### Was there a significant change to the year-on-year crime rate for violent crimes in neighborhoods of Chicago in a three year period before and after activation of the surveillance cameras in 2003?

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

- Dataset Name: Blue Light Cameras in the city of Chicago
- Link to the dataset: https://redshiftzero.github.io/policesurveillance/
- Number of observations: 715

The camera locations were indicated on a map. Therefore, the data we needed, that is the latitude and logitude of camera locations was embedded in HTML code with Javascript. We used Beautiful Soup along with python regular expressions to extract the data from source code of the link above. The script for extracting the data is as below. The latitude and logitude were saved in a csv file named 'lat_long.csv'. 

In [ ]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import re
import csv
import matplotlib 

cameras = 'https://redshiftzero.github.io/policesurveillance/pod.html'
data = requests.get(cameras)
soup = BeautifulSoup(data.content, 'html.parser')

right_table = soup.find_all('script')
what_we_need = right_table[6]

# Regular expression to find patterns of data we need
pattern = re.compile(".*var marker_.* = L.marker\(\[.*\n.*\n")
all_patterns = pattern.findall(what_we_need.string)

# Using string functions to convert data to needed form
with open('lat_long.csv', mode='w') as camera_file:
    camera_file = csv.writer(camera_file, delimiter=',', lineterminator = '\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    camera_file.writerow(["Latitude", "Longitude"])
    for each in all_patterns:
        each = each.strip()
        each = ' '.join(each.split())
        each = each[each.find('[') + 1:each.find(']')]
        (lat, long) = each.split(',')
        lat = lat.strip()
        long = long.strip()
        camera_file.writerow([lat, long])

# Setup

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab
import pandas as pd

from bs4 import BeautifulSoup
import requests
import gps_to_neighborhood

import statsmodels.api as sm
import patsy
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*